# Load and use model in Redis AI

1. docker run -p 6379:6379 -it --rm redisai/redisai


* https://oss.redislabs.com/redisai/commands/
* https://oss.redislabs.com/redisai/

Use Redisai.py to upload the model to Redis.

In [ ]:
import redisai
import ml2rt
from PIL import Image
from torchvision import transforms
import numpy as np

class Object(object): pass

In [ ]:
tr = Object()
tr.redis = redisai.Client()

In [ ]:
tr.redis.loadbackend('TORCH', 'redisai_torch/redisai_torch.so')

In [ ]:
tr.loaded_model = ml2rt.load_model('model.pt')

In [ ]:
tr.redis.modelset(
    "walk_or_run", 
    redisai.Backend.torch,
    redisai.Device.cpu,
    tr.loaded_model)

In [ ]:
!ls ../input/walk-or-run/walk_or_run_test/test/walk/

In [ ]:
input_path = "../input/walk-or-run"
tr.img = Image \
    .open(input_path + '/walk_or_run_test/test/walk/walk_9d193f21.png') \
    .convert("RGB")
tr.img

Our model expects images in a certain format:
   * 224 x 224
   * Colors re-scaled the mean and std dev:
     * mean=[0.485, 0.456, 0.406]
     * std=[0.229, .224, .225]
   * The image must also be a part of an array, even if it's just one image. We will insert another dimension as the beginning to achieve this.

In [ ]:
tr.transform = transforms.Compose([
    transforms.Resize(224, 224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, .224, .225])
])

In [ ]:
tr.img_t = tr.transform(tr.img)
tr.img_t = np.expand_dims(tr.img_t, axis=0)
tr.img_t.shape

Send the image to Redis AI:

In [ ]:
tr.redis.tensorset('input', tr.img_t)

Apply the model to the image:

In [ ]:
tr.redis.modelrun('walk_or_run', 'input', 'pred')

Get the prediction back:

In [ ]:
tr.redis.tensorget('pred')

This is prediction is from the Linear layer. To convert to probabilities, we need to apply the Softmax function:

In [ ]:
def softmax(v):
    return np.exp(v) / np.sum(np.exp(v))

In [ ]:
softmax(tr.redis.tensorget('pred'))

Our labels are ['run', 'walk']. So this result means the predicted probabilities are:
  * 'run' is 0.004, and 
  * 'walk' is 0.995